In [3]:
!pip install arxiv

  Using cached arxiv-2.1.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached feedparser-6.0.10-py3-none-any.whl.metadata (2.3 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached arxiv-2.1.0-py3-none-any.whl (11 kB)
Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)


In [23]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f9728ebebf0>)

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [6]:
retriever_tool.name

'langsmith_search'

In [7]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [8]:
tools=[wiki,arxiv,retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/josephmata/Documents/Updated-Langchain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f9728d71a20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorst

In [10]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [11]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [51]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, mess

In [13]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the 

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring you can ship quickly and with confidence. LangSmith offers Python and Typescript SDKs for all your development needs. You can create an API key, set up your environment, and log your first trace using LangSmith. \n\nLangSmith supports various workflows at each stage of the LLM application lifecycle and provides capabilities for tracing, evaluation, pricing, self-hosting, and proxy functionalities. Additionally, LangSmith offers a Prompt Hub for prompt management and a Cookbook with tutorials and walkthroughs. \n\nIf you have sensitive data that cannot be logged or need a private deployment of LangSmith, you can reach out to the LangChain team for self-hosting options. For more information, you can visit the LangSmith website or join their Discord community for discussions.

In [29]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}

In [24]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

from langchain.memory import ConversationBufferMemory #memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

loader=WebBaseLoader("https://www.realitymgmt.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

retriever_tool=create_retriever_tool(retriever,"realitycenter_search",
                      "Search for information about Reality Center. For any questions about Reality Center, you must use this tool!")
tools=[wiki,arxiv,retriever_tool]


### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True, memory=memory)

agent_executor.invoke({"input":"Tell me about Reality Center"})["output"]



> Entering new AgentExecutor chain...

Invoking: `realitycenter_search` with `{'query': 'Reality Center'}`


Reality Center | Sensory Wellness Therapy | Santa Monica




































top of pageJOIN OUR COMMUNITY AND UNLOCK ACCESS TO EXCLUSIVE OFFERS & EVENTS! CLICK HERELog InHOMEABOUTPROCESSBOOK EXPERIENCEPACKAGESINITIATIVESAffiliatesWorkplace WellnessSupport A VeteranGIFT CARDFAQCONTACTMoreUse tab to navigate through the menu items.WE CREATE TRANSFORMATIONSTATE OF THE ART SENSORY WELLNESS CENTER START YOUR JOURNEY FOR ONLY $59





UNLOCK YOUR HIGHEST POTENTIALReality Center is a state of the art sensory  wellness center and digital psychedelic lab based in Santa Monica, California. By combining ancient philosophies and modern science, we are able to address a wide array of mental, physical and emotional issues in a truly one of a kind experience.SNEAK PEEK - CLICK PLAY

Our Reality Managers are experts in the field and will make sure you have a safe and inspired

"Reality Center is a state-of-the-art sensory wellness center and digital psychedelic lab based in Santa Monica, California. They combine ancient philosophies and modern science to address a wide array of mental, physical, and emotional issues in a unique experience. The center offers sensory wellness therapy using frequency technologies to stimulate the nervous system's natural healing mechanisms. This therapy synchronizes the senses through light, sound, and vibration to induce deep states of relaxation and inspiration while providing a neurological reset. The experience at Reality Center includes laying on a vibrating liquid or foam bed with pulsing lights and synchronized sounds that take individuals on a deep journey into their consciousness. This experience aims to reduce stress and anxiety, improve breathing and blood flow, enhance creativity and inspiration, promote better sleep, provide pain relief, improve focus, release trauma, reset the nervous system, and promote self-love

In [22]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='

In [34]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

from langchain.memory import ConversationBufferMemory #memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


def search(input):
    loader=WebBaseLoader("https://www.realitymgmt.com/")
    docs=loader.load()
    documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
    vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
    vector = vectordb.similarity_search(input)
    return f"{vector[0].metadata['page']}"


tools=[wiki,arxiv,search]


### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True, memory=memory)

agent_executor.invoke({"input":"Tell me about Reality Center"})["output"]

KeyError: 'tools'

In [45]:
def search(input):
    loader=WebBaseLoader("https://www.realitymgmt.com/")
    docs=loader.load()
    documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
    vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
    vector = vectordb.similarity_search(input, k=4)
    return f"{vector}"
x =search('whats is reality center')

In [49]:
x

'[Document(page_content=\'Reality Center | Sensory Wellness Therapy | Santa Monica\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\ntop of pageJOIN OUR COMMUNITY AND UNLOCK ACCESS TO EXCLUSIVE OFFERS & EVENTS! CLICK HERELog InHOMEABOUTPROCESSBOOK EXPERIENCEPACKAGESINITIATIVESAffiliatesWorkplace WellnessSupport A VeteranGIFT CARDFAQCONTACTMoreUse tab to navigate through the menu items.WE CREATE TRANSFORMATIONSTATE OF THE ART SENSORY WELLNESS CENTER\\xa0START YOUR JOURNEY FOR ONLY $59\\n\\n\\n\\n\\n\\nUNLOCK YOUR HIGHEST POTENTIALReality Center is a state of the art sensory\\xa0 wellness center and digital psychedelic lab based in Santa Monica, California. By combining ancient philosophies and modern science, we are able to address a wide array of mental, physical and emotional issues in a truly one of a kind experience.SNEAK PEEK - CLICK PLAY\', metadata={\'source\': \'https://www.realitymgmt.com/\', \'title\': \'Reality Cente

In [47]:
x.metadata['page']

AttributeError: 'str' object has no attribute 'metadata'